In [1]:
# Standard Libraries
import os
import json
import datetime
import numpy as np
import tensorflow as tf

import time
import nvtx

import argparse

import math
from tensorflow.keras.utils import Progbar

# ------------------------------- CUSTOM FUNCTIONS ------------------------------------------------
# Custom Library
import sys
sys.path.append('../')
    
from proxy_apps.apps.timeseries_prediction import deepDMD, proxyDeepDMD, proxyDeepDMDMGPU

from proxy_apps.utils.tf import TimingCallback
from proxy_apps.utils.data.main import NpEncoder
from proxy_apps.utils import file_reader, path_handler
from proxy_apps.utils.data.grid import GridNetworkDataHandler, GridNetworkTFDataHandler, GridNetworkNewGen, TransientDataset

In [2]:
# System Setup
config = file_reader.read_config()

_N_EPOCHS = 2
_BATCH_SIZE = 65536
_APP_NAME = config["info"]["app_name"]
_NROWS = int(config["data"]["n_rows"])
_NCOLS = int(config["data"]["n_cols"])
_REPEAT_COLS = int(config["data"]["repeat_cols"])
_WINDOW_SIZE = int(config["data"]["window_size"])
_SHIFT_SIZE = int(config["data"]["shift_size"])
_STRIDE = int(config["data"]["stride"])
_N_SIGNALS = int(config["data"]["n_signals"])

_DTYPE = config["model"]["dtype"]

_LABEL = "TFDataOpt"
_SUFFIX =  "gpu" + '_' + \
            "a100" + '_' + \
            'ng' + str(1) + '_' + \
            'nc' + str(-1) + '_' + \
            'e' + str(_N_EPOCHS) + '_' + \
            'b' + str(_BATCH_SIZE) + '_' + \
            'r' + str(_REPEAT_COLS) + '_' + _LABEL

performance_dict = dict()

tic = time.time()

# current directory
curr_dir = "./"

# output directory
output_dir = path_handler.get_absolute_path(curr_dir, config["info"]["output_dir"] + config["info"]["name"] + "/" + config["info"]["app_name"] + "/" + _DTYPE + "/R" + str(_REPEAT_COLS) + "/")
if not os.path.exists(output_dir): os.makedirs(output_dir)

# TensorFlow Setup
print("[INFO] Tensorflow version: ", tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

[INFO] Tensorflow version:  2.4.0


In [3]:
def benchmark(dataset, num_epochs=2, steps=10):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for s, sample in enumerate(dataset):
            # Performing a training step
            time.sleep(0.01)
            if s == steps - 1:
                break
            
    print("Execution time:", time.perf_counter() - start_time)

In [4]:
# tf.compat.v1.disable_eager_execution()
print("[INFO] Eager mode: ", tf.executing_eagerly()) # For easy reset of notebook state.

tf.keras.backend.clear_session()
tf.keras.backend.set_floatx(_DTYPE)

[INFO] Eager mode:  True


In [5]:
def get_indexer(n_rows, window_size, shift_size, start_point, leave_last):
    return np.arange(window_size)[None, :] + start_point + shift_size*np.arange(((n_rows - window_size - leave_last - start_point) // shift_size) + 1)[:, None]

In [6]:
data_handler = GridNetworkDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                        n_rows=_NROWS,
                                        n_cols=_NCOLS,
                                        repeat_cols=_REPEAT_COLS,
                                        dtype=_DTYPE
                                     ) 

scenario_data = data_handler.load_grid_data()
X_data, Y_data, U_data, V_data, Yp, Yf = data_handler.create_windows(scenario_data)
X_array, Y_array, U_array, V_array, Yp_array, Yf_array = data_handler.scale_data(X_data, Y_data, U_data, V_data, Yp, Yf)

[INFO]: Loading the datasets from the directory: /qfs/people/jain432/pacer/data/TrainingDataIEEE68bus
[INFO]: Loading data for 30 scenarios ...
[INFO]: Total number of scenarios loaded: 30
[INFO]: Shape of each scenario loaded:  (1400, 136)
[INFO]: Done ...
[INFO]: Original dataset size: 1400
[INFO]: Chosen dataset size: 800
[INFO]: Length of X_data:  1800
[INFO]: Length of each window after down sampling:  (800, 136)
[INFO]: Yp_array shape:  (41970, 136)
[INFO]: Yf_array shape:  (41970, 136)
[INFO]: X_array shape:  (1440000, 136)
[INFO]: Y_array shape:  (1440000, 136)
[INFO]: U_array shape:  (1440000, 136)
[INFO]: V_array shape:  (1440000, 136)


In [7]:
data_handler = GridNetworkNewGen(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                        n_rows=_NROWS,
                                        n_cols=_NCOLS,
                                        repeat_cols=_REPEAT_COLS,
                                        d_type=_DTYPE
                                     )

x_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 0, _N_SIGNALS)
y_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 1, 0)

scenario_data = data_handler.get_training_data(x_indexer, y_indexer)

In [8]:
X_array[0, :]

array([ 0.08234169,  0.08398943,  0.08258068,  0.08214683,  0.08244791,
        0.08239396,  0.0825465 ,  0.08293292,  0.08758576,  0.08190964,
        0.08206848,  0.08195436,  0.08184469,  0.08165109,  0.08036628,
        0.0798736 ,  0.08542501,  0.08431926,  0.04962437,  0.03824485,
        0.08420936,  0.09290614,  0.09679138,  0.08237581,  0.08692641,
        0.11023701,  0.09832751,  0.14486176,  0.15468656,  0.08604157,
        0.08542665,  0.09650717,  0.09626098,  0.09627406,  0.09861494,
        0.09202703,  0.09158971,  0.09184748,  0.09850423,  0.03992778,
       -0.0330755 , -0.0259497 ,  0.09590663,  0.09609054,  0.10489531,
        0.10175143,  0.06886591,  0.05863196,  0.1120856 ,  0.12620922,
        0.1111895 ,  0.14195852,  0.08204857,  0.08240602,  0.08138131,
        0.03664793,  0.02118596,  0.09608962,  0.12064654,  0.07192818,
        0.17851271,  0.07751198,  0.10213777,  0.09318556,  0.09092183,
       -0.03392203, -0.02954702,  0.15240774,  0.51401846,  0.45

In [9]:
for x, y in scenario_data.take(1):
    print(x)

tf.Tensor(
[-0.12206114 -0.13795394 -0.13999129 -0.13877701 -0.13675519 -0.13687104
 -0.13526483 -0.13446715 -0.12152589 -0.13873522 -0.13813075 -0.13862319
 -0.13911997 -0.14013874 -0.14458367 -0.14651475 -0.14420605 -0.14259088
 -0.15075401 -0.15236286 -0.14850716 -0.15057178 -0.15054028 -0.14708136
 -0.14027052 -0.14327165 -0.14314481 -0.14571935 -0.14655097 -0.12253022
 -0.12711685 -0.12019089 -0.12233241 -0.12326352 -0.12609909 -0.11877959
 -0.11593057 -0.12876138 -0.12529942 -0.07787391  0.00333725 -0.02453103
 -0.12180074 -0.12204847 -0.13365506 -0.13630499 -0.10730083 -0.09660477
 -0.14428683 -0.15993669 -0.14141032 -0.18326793 -0.13871331 -0.13854983
 -0.13982429 -0.15246719 -0.15453239 -0.15269285 -0.1530478  -0.14339387
 -0.14837856 -0.13988188 -0.1158767  -0.11910663 -0.11320636  0.00489531
 -0.02173361 -0.19421897  0.63451151  0.5317788   0.33944751  0.06799646
  0.08201882  0.10255614  0.00699209  0.00289895  0.42210129  0.19097658
  0.14857335  0.56093779  0.16398019  0.

In [ ]:
# ------------------------------- DATA LOADING ------------------------------------------------   
l_start = time.time()
if _LABEL == "Baseline":
    data_handler = GridNetworkDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

    scenario_data = data_handler.load_grid_data()
elif _LABEL == "TFDataOptPrev":
    data_handler = GridNetworkTFDataHandler(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            dtype=_DTYPE
                                         ) 

    scenario_data = data_handler.load_grid_data()
elif _LABEL == "TFDataOpt":
    data_handler = GridNetworkNewGen(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
                                            n_rows=_NROWS,
                                            n_cols=_NCOLS,
                                            repeat_cols=_REPEAT_COLS,
                                            d_type=_DTYPE
                                         )

    x_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 0, _N_SIGNALS)
    y_indexer = get_indexer(_NROWS, _WINDOW_SIZE, _SHIFT_SIZE, 1, 0)

    scenario_data = data_handler.get_training_data(x_indexer, y_indexer)

l_stop = time.time()
print('[INFO]: Time taken for loading datasets:', l_stop - l_start, 'seconds')

In [ ]:
# window_size = 800
# shift_size = 10
# stride = 1
# N = 3

# x_indexer = get_indexer(_NROWS, window_size, shift_size, 0, 3)
# y_indexer = get_indexer(_NROWS, window_size, shift_size, 1, 0)

# training_data = data_handler.get_training_data(x_indexer, y_indexer)
# benchmark(training_data)

In [ ]:
if _LABEL in ["Baseline", "TFDataOptPrev"]:
    # ------------------------------- DATA PREPROCESSING ------------------------------------------------
    i_start = time.time()
    X_data, Y_data, U_data, V_data, Yp, Yf = data_handler.create_windows(scenario_data)
    i_stop = time.time()
    print('[INFO]: Time taken for creating X datasets:', i_stop - i_start, 'seconds')
    
    # ------------------------------- DATA NORMALIZATION ------------------------------------------------
    n_start = time.time()
    X_array, Y_array, U_array, V_array, Yp_array, Yf_array = data_handler.scale_data(X_data, Y_data, U_data, V_data, Yp, Yf)
    n_stop = time.time()
    print('[INFO]: Time taken for normalization:', n_stop - n_start, 'seconds')

In [ ]:
# timing callback
timing_cb = TimingCallback()

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mse')
model.build(input_shape=(None,136))
model.summary()

In [ ]:
np.__version__

In [ ]:
if _LABEL == "Baseline":
    m_start = time.time()
    model.fit(X_array, Y_array, epochs=2, batch_size=_BATCH_SIZE, callbacks=[timing_cb])
    m_stop = time.time()
elif _LABEL == "TFDataOptPrev":
    # tensorflow dataset conversion
    tf_conv_start = time.time()
    training_dataset = tf.data.Dataset.zip((X_array, Y_array)).batch(_BATCH_SIZE)
    training_dataset = training_dataset.cache()
    training_dataset = training_dataset.shuffle(buffer_size=22)
    training_dataset = training_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    tf_conv_stop = time.time()
    print('[INFO]: Tensorflow Conversion Time:', tf_conv_stop - tf_conv_start, 'seconds')
    
    m_start = time.time()
    model.fit(training_dataset, epochs=_N_EPOCHS, callbacks=[timing_cb])
    m_stop = time.time()
elif _LABEL == "TFDataOpt":
    training_dataset = scenario_data.batch(_BATCH_SIZE)
    training_dataset = training_dataset.cache()
    training_dataset = training_dataset.shuffle(buffer_size=x_indexer.shape[0]*x_indexer.shape[1]*len(data_handler.dir_list)//_BATCH_SIZE)
    training_dataset = training_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    m_start = time.time()
    model.fit(training_dataset, epochs=_N_EPOCHS, callbacks=[timing_cb], workers=64, use_multiprocessing=True)
    m_stop = time.time()
    
# print info
print('[INFO]: Time taken for model training (time module):', m_stop - m_start, 'seconds')
print('[INFO]: Time taken for model training (Keras):', sum(timing_cb.logs), 'seconds')

In [ ]:
# window_size = 800
# shift_size = 10
# stride = 1
# N = 3

# data_handler_seqgen = GridNetworkSequentialGen(scenario_dir=path_handler.get_absolute_path(curr_dir, config["info"]["input_dir"]),
#                                             n_rows=_NROWS,
#                                             n_cols=_NCOLS,
#                                             repeat_cols=_REPEAT_COLS,
#                                             d_type=_DTYPE
#                                          )
# trimmed_scenarios_seqgen = data_handler_seqgen.load_grid_data()
# training_data_seqgen = data_handler_seqgen.get_training_data(trimmed_scenarios_seqgen, window_size, shift_size, stride, N, 1)
# benchmark(training_data_seqgen)